# Algorytmy wyszukiwania wzorców

In [271]:
import matplotlib.pyplot as plt
from time import time
from tabulate import tabulate

## Algorytm naiwny

In [264]:
def naive_string_matching(text, pattern):
    pattern_len = len(pattern)
    result = []
    for i in range(len(text) - pattern_len + 1):
        if text[i:i+pattern_len] == pattern:
            result.append(i)
    return(result)

## Algorytm automatu skończonego

In [ ]:
def transition_table(pattern):
    result = []
    for q in range(0, len(pattern) + 1):
        result.append({})
        for a in list(set(pattern)):
            k = min(len(pattern) + 1, q + 2)
            while True:
                k = k - 1

                if(k == 0 or pattern[:k] == (pattern[:q] + a)[-k:]):
                    break
            result[q][a] = k
    return result


def fa_string_matching(text, delta):
    result = []
    q = 0
    length = len(delta) - 1
    for i in range(0, len(text)):
        q = delta[q].get(text[i], 0)
        if(q == length):
            result.append(i + 1 - q)
    return result

## Algorytm Knutha-Morrisa-Pratta

In [ ]:
def prefix_function(pattern):
    pi = [0]
    k = 0
    for q in range(1, len(pattern)):
        while(k > 0 and pattern[k] != pattern[q]):
            k = pi[k-1]
        if(pattern[k] == pattern[q]):
            k = k + 1
        pi.append(k)
    return pi


def kmp_string_matching(text, pattern,  pi):
    result = []
    q = 0
    for i in range(0, len(text)):
        while(q > 0 and pattern[q] != text[i]):
            q = pi[q-1]
        if(pattern[q] == text[i]):
            q = q + 1
        if(q == len(pattern)):
            result.append(i+1-q)
            q = pi[q-1]
    return result

In [397]:
# Funkcja zwracająca czasy wykonania poszczególnych algorytmów
#(czas pre-processingu)
def test(text, pattern):
    start = time()
    naive_string_matching(text, pattern)
    naive_time = time() - start
    
    delta = transition_table(pattern)
    start = time()
    fa_string_matching(text, delta)
    fa_time = time() - start
    
    pi = prefix_function(pattern)
    start = time()
    kmp_string_matching(text, pattern, pi)
    kmp_time = time() - start
    
    
    return naive_time, fa_time, kmp_time

## Znajdowanie wzorca "art" w danej ustawie

In [423]:
with open("1997_714.txt", "r") as file:
    text = file.read()
    test_results = test(text, "art")
    print(tabulate({"Algorithm" : ["Naive", "Finite Automata", "KMP"],
                "Time [s]" : test_results}, headers="keys"))

Algorithm          Time [s]
---------------  ----------
Naive             0.0667269
Finite Automata   0.0344884
KMP               0.0435891


## Znajdowanie wzorca "Ukraina" we fragmencie wikipedi

In [421]:
with open("passages-head.tsv", "r") as file:
    text = file.read()
    test_results = test(text, "Ukraina")
    print(tabulate({"Algorithm" : ["Naive", "Finite Automata", "KMP"],
                "Time [s]" : test_results}, headers="keys"))

Algorithm          Time [s]
---------------  ----------
Naive               16.69
Finite Automata     14.7412
KMP                 15.0942


## Przypadek dla którego czasy działania algorytmów 2. i 3. są 5-krotnie krótsze od algorytmu naiwnego

In [417]:
text = "aaaaaaaaa"*1000+'b'
test_results = test(text, "aaaaaaaaa"*200+'b')
print(tabulate({"Algorithm" : ["Naive", "Finite Automata", "KMP"],
                "Time [s]" : test_results}, headers="keys"))

Algorithm          Time [s]
---------------  ----------
Naive            0.00714231
Finite Automata  0.00165939
KMP              0.00352836


## Przypadek dla którego czas obliczenia tablicy przejścia automatu skończonego jest 5-krotnie dłuższy od czasu utworzenia funkcji przejścia w algorytmie KMP

In [420]:
pattern = "qwertyuiopasdfghjklzxcvbnm"
start = time()
transition_table(pattern)
transition_time = time() - start

start = time()
prefix_function(pattern)
prefix_time = time() - start

print(tabulate({"Name" : ["Transition table", "Prefix function"],
                "Time [s]" : [transition_time, prefix_time]}, headers="keys"))

Name                 Time [s]
----------------  -----------
Transition table  0.0171127
Prefix function   0.000234604
